In [28]:
import numpy as np
import pandas as pd

In [29]:
df = pd.read_csv('source/raw-news/df_5.csv', encoding='utf-8')

# 新闻数据清洗

去掉重复

In [30]:
df.drop_duplicates(inplace=True)

In [31]:
print(df.columns.tolist())

['CREAT_TM', 'UPDT_TM', 'IS_VLD', 'F4000', 'F4001', 'F4003', 'F4004', 'F4020', 'F4022', 'F4023', 'F4024', 'F4025', 'F4026', 'F4027']


In [32]:
rename_map = {
    "F4000": "news_id",
    "F4003": "publish_time",
    "F4001": "title",
    "F4004": "source",
    "F4020": "content",
    "F4022": "news_type",
    "F4023": "source_id",
    "F4024": "info_source",
    "F4025": "source_category",
    "F4026": "url",
    "F4027": "summary"
}

df = df.rename(columns=rename_map)


In [33]:
print(df.head())

              CREAT_TM              UPDT_TM  IS_VLD  news_id  \
0  2025-04-02 16:07:33  2025-04-02 16:07:33       1        3   
1  2025-04-02 16:07:33  2025-04-02 16:07:33       1        4   
2  2025-04-02 16:07:33  2025-04-02 16:07:33       1        5   
3  2025-04-02 16:07:33  2025-04-02 16:07:33       1        6   
4  2025-04-02 16:07:33  2025-04-02 16:07:33       1        7   

                              title         publish_time       source  \
0            3月基金发行回暖 单月发行份额超1000亿份  2025-04-02 15:45:02        证券时报网   
1     4月首缕“春风”吹到医药股：全线爆发 后续行情持续性几何？  2025-04-02 15:44:02     21世纪经济报道   
2      联博智远混合将于4月7日发售 以均衡策略把握A股长期机遇  2025-04-02 15:30:47    中国证券报·中证网   
3  ETF今日收评 | 新兴亚洲ETF涨超5% 新经济ETF跌超5%  2025-04-02 15:17:56       每日经济新闻   
4                   联博基金旗下第二只产品即将发行  2025-04-02 14:45:12  上海证券报·中国证券网   

                                             content  news_type  \
0  股市回暖背景下，权益类基金发行拾级而上。公募排排网数据显示，截至3月31日，按基金成立日统计...          2   
1  医药终于领涨了。二季度开局，医药股一马当先展现出强势上涨姿态。4月1日，医药生

In [34]:
import test_qwen

In [35]:
test_qwen.test_dify_connectivity()
    
    # 2. 调用封装好的函数
print("\n--- [2] 正在调用封装后的 call_dify_api ---")
test_query = "你好，请确认你是通过封装后的函数调用的，并简要自我介绍。"
    
result = test_qwen.call_dify_api(test_query)
    
if "answer" in result:
    print("\n✅ 调用成功!")
    print(f"模型回复: {result['answer']}")
    print(f"Conversation ID: {result.get('conversation_id')}")
else:
    print("\n❌ 调用失败或返回异常")
    print(f"结果详情: {json.dumps(result, ensure_ascii=False, indent=2)}")


--- [1] 正在测试 Dify 网络连通性 ---
基础 URL 请求状态码: 404

--- [2] 正在调用封装后的 call_dify_api ---

✅ 调用成功!
模型回复: 你好！我是通义千问（Qwen），由通义实验室研发的大语言模型。我并非通过封装后的函数调用，而是直接以模型身份与你进行对话。我的设计目标是理解并生成自然语言，能够回答问题、创作文字、提供信息查询等。如果你有任何问题或需要帮助，欢迎随时告诉我！
Conversation ID: ef201dd5-87a5-4fff-9b54-4d7230726931


In [36]:
test_query = "你好，请确认你是通过封装后的函数调用的，并简要自我介绍。"
    
result = test_qwen.call_dify_api(test_query)
print(f"模型回复: {result['answer']}")
print(f"Conversation ID: {result.get('conversation_id')}")

模型回复: 你好！我是通义千问（Qwen），由通义实验室研发的大语言模型。我并非通过封装后的函数调用，而是直接以模型身份与你进行对话。我的设计目标是理解并生成人类语言，能够回答问题、创作文字、提供信息查询等。如果你有任何问题或需要帮助，欢迎随时告诉我！
Conversation ID: 56a8a74a-0dc0-497c-afbc-4fe75913caf1


In [38]:
!pip install rapidfuzz

  Using cached rapidfuzz-3.14.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
Using cached rapidfuzz-3.14.3-cp312-cp312-macosx_11_0_arm64.whl (1.4 MB)


In [39]:
import re
import json
import pandas as pd
from rapidfuzz import process, fuzz

# ----------------------------
# 0) 加载你的三张词表
# ----------------------------
def load_taxonomies(
    company_csv: str = "source/industry/company-listcsv.csv",
    concept_csv: str = "source/industry/concept-list.csv",
    industry_csv: str = "source/industry/industry-list.csv",
):
    df_c = pd.read_csv(company_csv)
    df_k = pd.read_csv(concept_csv)
    df_i = pd.read_csv(industry_csv)

    # 兼容列名：FullName / 中文全称
    if "FullName" in df_c.columns:
        name_col = "FullName"
    elif "中文全称" in df_c.columns:
        name_col = "中文全称"
    else:
        raise ValueError(f"公司表找不到 FullName/中文全称 列，实际列：{df_c.columns.tolist()}")

    # 兼容股票代码列：Symbol / 股票代码
    if "Symbol" in df_c.columns:
        code_col = "Symbol"
    elif "股票代码" in df_c.columns:
        code_col = "股票代码"
    else:
        code_col = None  # 不强制

    companies = []
    for _, r in df_c.iterrows():
        full = str(r[name_col]).strip()
        if not full or full == "nan":
            continue
        companies.append({
            "full_name": full,
            "code": (str(r[code_col]).strip() if code_col and pd.notna(r[code_col]) else None)
        })

    concepts = []
    for _, r in df_k.iterrows():
        concepts.append({
            "code": str(r["conceptCode"]).strip(),
            "name": str(r["conceptName"]).strip()
        })

    industries = []
    for _, r in df_i.iterrows():
        industries.append({
            "code": str(r["industryCode"]).strip(),
            "name": str(r["industryName"]).strip(),
            "level": int(r["industryLevel"]) if "industryLevel" in df_i.columns and pd.notna(r["industryLevel"]) else None
        })

    return companies, concepts, industries


# ----------------------------
# 1) 公司名归一化：用来把简称映射回全称
# ----------------------------
_COMMON_SUFFIX = [
    "股份有限公司", "有限公司", "集团股份有限公司", "集团有限公司",
    "集团", "股份", "有限责任公司", "公司"
]

def normalize_company_name(s: str) -> str:
    s = re.sub(r"\s+", "", s)
    # 去掉常见后缀
    for suf in _COMMON_SUFFIX:
        if s.endswith(suf) and len(s) > len(suf):
            s = s[:-len(suf)]
            break
    # 去括号内容（有些简称带括号）
    s = re.sub(r"[（(].*?[）)]", "", s)
    return s


def build_company_index(companies):
    """
    建两个索引：
    - full_name_set：用于直接全称命中
    - norm_to_full：归一名 -> 多个全称（可能冲突）
    - norm_list：所有归一名列表（用于模糊匹配）
    """
    full_name_set = set()
    norm_to_full = {}
    for c in companies:
        full = c["full_name"]
        full_name_set.add(full)
        norm = normalize_company_name(full)
        norm_to_full.setdefault(norm, []).append(c)

    norm_list = list(norm_to_full.keys())
    return full_name_set, norm_to_full, norm_list


# ----------------------------
# 2) 本地抽行业/概念：直接词典匹配（简单稳）
# ----------------------------
def dict_match(text: str, items, key_name="name", max_hits=20):
    hits = []
    for it in items:
        name = it[key_name]
        if name and name in text:
            hits.append(it)
            if len(hits) >= max_hits:
                break
    return hits


# ----------------------------
# 3) 本地抽人名（粗召回）：职位触发词 + 2~4字中文名
# ----------------------------
_TITLE_TRIG = r"(董事长|总经理|CEO|CFO|总裁|行长|董事|副总经理|负责人|发言人|总监|首席科学家|院士)"
def extract_persons_rule(text: str, max_hits=10):
    persons = []
    for m in re.finditer(_TITLE_TRIG, text):
        start = max(0, m.start() - 6)
        end = min(len(text), m.end() + 10)
        window = text[start:end]
        # 在职位附近找 2~4 个汉字的人名
        nm = re.search(r"([\u4e00-\u9fff]{2,4})" + m.group(0), window)
        if nm:
            persons.append({
                "name": nm.group(1),
                "title": m.group(0),
                "organization": None,
                "relevance": 0.5,
                "evidence": [window]
            })
        if len(persons) >= max_hits:
            break
    return persons


# ----------------------------
# 4) 模型辅助：只让模型吐出“表面名/surface forms”
#    不让它做映射（映射交给本地，避免胡编）
# ----------------------------
def llm_suggest_surface_entities(call_dify_api, content: str):
    prompt = f"""
你是信息抽取器。只基于文本，抽取“表面出现”的实体，不要猜测不在文本里的信息。
输出严格 JSON（不要多余文字），格式如下：
{{
  "company_surface": ["在文本中出现的公司简称/全称/机构名", "..."],
  "person_surface": ["在文本中出现的人名", "..."]
}}

文本：
{content}
""".strip()

    resp = call_dify_api(prompt)
    ans = resp.get("answer", "")
    try:
        return json.loads(ans)
    except Exception:
        return {"company_surface": [], "person_surface": []}


# ----------------------------
# 5) 把 surface company 映射回你公司全称（核心）
# ----------------------------
def map_surface_to_full(surface_names, full_name_set, norm_to_full, norm_list,
                        score_threshold=88, topk=3):
    """
    先：如果 surface 直接就是全称 -> 命中
    再：归一化 exact -> 命中
    再：模糊匹配到最像的归一名 -> 命中（阈值控制）
    """
    mapped = []
    seen_full = set()

    for s in surface_names:
        if not s or s == "nan":
            continue
        s = str(s).strip()
        if len(s) < 2:
            continue

        # 1) 全称直接命中
        if s in full_name_set:
            cand_list = [c for c in norm_to_full.get(normalize_company_name(s), []) if c["full_name"] == s]
            c = cand_list[0] if cand_list else {"full_name": s, "code": None}
            if c["full_name"] not in seen_full:
                mapped.append((c, 1.0, s))
                seen_full.add(c["full_name"])
            continue

        # 2) 归一化 exact
        sn = normalize_company_name(s)
        if sn in norm_to_full:
            # 如果同一个归一名对应多个公司，就先都放进去（或者你可再加 disambiguation）
            for c in norm_to_full[sn]:
                if c["full_name"] not in seen_full:
                    mapped.append((c, 0.95, s))
                    seen_full.add(c["full_name"])
            continue

        # 3) 模糊匹配到“归一名”
        best = process.extractOne(sn, norm_list, scorer=fuzz.WRatio)
        if best:
            best_norm, best_score, _ = best
            if best_score >= score_threshold:
                for c in norm_to_full[best_norm][:topk]:
                    if c["full_name"] not in seen_full:
                        mapped.append((c, best_score / 100.0, s))
                        seen_full.add(c["full_name"])

    return mapped


# ----------------------------
# 6) 最终：从 df['content'] 抽 entities（你要的主函数）
# ----------------------------
def extract_entities_from_content(
    call_dify_api,
    content: str,
    companies, concepts, industries,
    full_name_set, norm_to_full, norm_list,
):
    # ① 本地：行业/概念（直接词典命中）
    concept_hits = dict_match(content, concepts, key_name="name")
    industry_hits = dict_match(content, industries, key_name="name")

    # ② 本地：公司全称直接命中（很少，但免费）
    full_hits = [c for c in companies if c["full_name"] in content][:30]

    # ③ 本地：人名规则召回
    persons_rule = extract_persons_rule(content)

    # ④ 模型：补公司简称/机构表面名、人名表面名
    sug = llm_suggest_surface_entities(call_dify_api, content)
    company_surface = list(dict.fromkeys(sug.get("company_surface", [])))  # 去重保序
    person_surface = list(dict.fromkeys(sug.get("person_surface", [])))

    # ⑤ 映射：简称 -> 全称（关键）
    mapped = map_surface_to_full(company_surface, full_name_set, norm_to_full, norm_list)

    # ⑥ 拼 entities（保持你之前 JSON 格式）
    entities = {
        "companies": [],
        "persons": [],
        "industries": [],
        "concepts": []
    }

    # 公司：合并 full_hits + mapped
    # full_hits 也按相同结构输出
    seen = set()
    for c in full_hits:
        if c["full_name"] in seen:
            continue
        entities["companies"].append({
            "name": c["full_name"],
            "code": c.get("code"),
            "role": "related",
            "relevance": 0.9,
            "mentions": content.count(c["full_name"]),
            "evidence": [c["full_name"]]
        })
        seen.add(c["full_name"])

    for c, rel, surface in mapped:
        if c["full_name"] in seen:
            continue
        entities["companies"].append({
            "name": c["full_name"],
            "code": c.get("code"),
            "role": "related",
            "relevance": float(rel),
            "mentions": content.count(surface) if surface else None,
            "evidence": [surface]
        })
        seen.add(c["full_name"])

    # 人物：规则 + 模型表面名（表面名也先塞进去，后续你可再清洗）
    seen_p = set()
    for p in persons_rule:
        k = (p["name"], p["title"])
        if k in seen_p:
            continue
        entities["persons"].append(p)
        seen_p.add(k)

    for pn in person_surface[:10]:
        if not pn or len(pn) < 2:
            continue
        entities["persons"].append({
            "name": pn,
            "title": None,
            "organization": None,
            "relevance": 0.4,
            "evidence": [pn]
        })

    # 行业/概念
    for it in industry_hits:
        entities["industries"].append({
            "name": it["name"],
            "code": it["code"],
            "relevance": 0.8
        })

    for it in concept_hits:
        entities["concepts"].append({
            "name": it["name"],
            "code": it["code"],
            "relevance": 0.8
        })

    return entities


In [40]:
# 1) 载入词典
companies, concepts, industries = load_taxonomies(
    "source/industry/company-listcsv.csv",
    "source/industry/concept-list.csv",
    "source/industry/industry-list.csv"
)

# 2) 建公司索引
full_name_set, norm_to_full, norm_list = build_company_index(companies)

# 3) 单条测试（只用 content）
content = df.loc[0, "content"]

entities = extract_entities_from_content(
    call_dify_api=test_qwen.call_dify_api,   # 你现有接口
    content=content,
    companies=companies,
    concepts=concepts,
    industries=industries,
    full_name_set=full_name_set,
    norm_to_full=norm_to_full,
    norm_list=norm_list
)

print(json.dumps({"entities": entities}, ensure_ascii=False, indent=2))


{
  "entities": {
    "companies": [
      {
        "name": "深圳市中金岭南有色金属股份有限公司",
        "code": "000060",
        "role": "related",
        "relevance": 0.9,
        "mentions": 2,
        "evidence": [
          "中金公司"
        ]
      },
      {
        "name": "东方证券股份有限公司",
        "code": "600958",
        "role": "related",
        "relevance": 0.95,
        "mentions": 1,
        "evidence": [
          "东方证券"
        ]
      }
    ],
    "persons": [],
    "industries": [
      {
        "name": "机器人",
        "code": "640701",
        "relevance": 0.8
      }
    ],
    "concepts": [
      {
        "name": "人工智能",
        "code": "CLS80207",
        "relevance": 0.8
      },
      {
        "name": "REITs",
        "code": "CLS80464",
        "relevance": 0.8
      },
      {
        "name": "人形机器人",
        "code": "CLS82064",
        "relevance": 0.8
      }
    ]
  }
}
